# Projeto de Pipeline de Dados do Telegram

Este projeto visa relacionar o mundo de chatbots em aplicativos como o Telegram com computação em nuvem. No caso, é realizada uma etapa transacional para capturar dados da API do Telegram de bots pelo AWS API Gateway, e a partir disto uma etapa analítica realizada em nuvem na AWS com três etapas: ingestão, ETL e apresentação. Por fim, no AWS Athena são feitas consultas para estimar quantidade de mensagens por dia no grupo do Telegram, quantidade de mensagens por hora e dia da semana, e média de tamanho das mensagens.

# 1. Análise Exploratória de Dados

A Telegram Bot API permite a comunicação de informações de mensagens, usuários e *bots* de um grupo com códigos externos. Existem alguns métodos que podem ser usados no Python para recuperar essas informações em formato JSON. Um deles é o método **getMe** que retorna informações sobre o *bot* do grupo. Já o método **getUpdates** retorna todas as informações com listas das mensagens mandadas em um grupo, disponíveis por até 24 horas. Para o acesso, é necessário um API token relacionado ao *bot*. Isso envolve usar linhas de código do tipo:

> base_url = f'https://api.telegram.org/bot{token}'
> 
> response = requests.get(url=f'{base_url}/getMe')

onde *token* é o API token do *bot* obtido do Telegram.


In [1]:
import os
import json

In [2]:
with open('/kaggle/input/data-getupdates/getMe.json', mode='r', encoding='utf8') as f:
    message = json.loads(f.read())
    print(json.dumps(
        message, indent=2
    ))

{
  "ok": true,
  "result": {
    "id": 6654125182,
    "is_bot": true,
    "first_name": "m42_ebac_bot",
    "username": "m42_pipeline_bot",
    "can_join_groups": false,
    "can_read_all_group_messages": false,
    "supports_inline_queries": false
  }
}


No método **getUpdates** temos informações mais relevantes das mensagens. Algumas chaves são **obrigatórias**, por exemplo first_name, is_bot, date, text. Mas, outras chaves são **opcionais** como last_name e username. Com um arquivo JSON de resposta do **getUpdates** armazenado como exemplo, podemos lê-lo para ver seu formato:

In [3]:
with open('/kaggle/input/data-getupdates/telegram.json', mode='r', encoding='utf8') as f:
    message = json.loads(f.read())
    print(json.dumps(
        message, indent=2
    ))

{
  "ok": true,
  "result": [
    {
      "update_id": 187921657,
      "message": {
        "message_id": 3,
        "from": {
          "id": 479372888,
          "is_bot": false,
          "first_name": "Mateus",
          "last_name": "Miguel",
          "username": "mateusmmiguel"
        },
        "chat": {
          "id": -4055988830,
          "title": "M42 Ebac Group",
          "type": "group",
          "all_members_are_administrators": true
        },
        "date": 1697323306,
        "text": "Ol\u00e1, mundo!"
      }
    },
    {
      "update_id": 187921658,
      "message": {
        "message_id": 4,
        "from": {
          "id": 479372888,
          "is_bot": false,
          "first_name": "Mateus",
          "last_name": "Miguel",
          "username": "mateusmmiguel"
        },
        "chat": {
          "id": -4055988830,
          "title": "M42 Ebac Group",
          "type": "group",
          "all_members_are_administrators": true
        },
        "date"

Como o arquivo JSON é semi-estruturado, a ideia é fazer um *data wrangling* para extrair apenas as informações obrigatórias das mensagens e armazená-las em um formato estruturado, tabular. Com isso, vai ser possível usar o AWS Athena para consultas SQL padronizadas, sobre a tabela 'telegram'. A função *parse_data* é capaz de fazer isto, basicamente percorrendo as chaves e listas 

In [4]:
def parse_data(data: dict) -> dict:
    # Função que faz o data wrangling para ir dos arquivos JSON para o .parquet diário
    parsed_data = dict()
    
    for key, value in data.items():
        if key == 'from':
            for k, v in data[key].items():
                if k in ['id', 'is_bot', 'first_name']:
                    parsed_data[f'user_{k}'] = [v]
        
        elif key == 'chat':
            for k, v in data[key].items():
                if k in ['id', 'type']:
                    parsed_data[f'chat_{k}'] = [v]
                    
        elif key in ['message_id', 'text']:
            parsed_data[key] = [value]
            
        elif key == 'date':
            tzinfo = timezone(offset=timedelta(hours=-3))
            parsed_data[key] = [value]
            parsed_data['timestamp'] = [datetime.fromtimestamp(value, tzinfo).strftime('%Y-%m-%d %H:%M:%S')]
            
    if not 'text' in parsed_data.keys():
        parsed_data['text'] = ['']
        
    return parsed_data